<a href="https://colab.research.google.com/github/V-Sekai-fire/meshgpt-pytorch/blob/main/MeshGPT_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We use a local runtime. https://research.google.com/colaboratory/local-runtimes.html

In [48]:
#!bash <(curl -L micro.mamba.pm/install.sh) 
#Prefix location? [~/micromamba] /workspace/micromamba
#!pip3 install notebook jupyterlab
#!micromamba create -n py311_meshgpt python=3.11 anaconda -c pytorch -c conda-forge -c anaconda -c defaults -y
#!micromamba activate py311_meshgpt
#!jupyter notebook --port=9999  --NotebookApp.port_retries=0 --allow-root
# Login with kernel
!pip3 install git+https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git
!pip3 install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
!pip3 install trimesh

  Cloning https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git to /tmp/pip-req-build-nm9bfsnl
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git /tmp/pip-req-build-nm9bfsnl
  Resolved https://github.com/MarcusLoppe/classifier-free-guidance-pytorch.git to commit 698c83562f8859c763880f200b210ff1081efedc
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-0_bnih0k
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-0_bnih0k
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit d35228edc46b550dc8faeefb832db7bd43a23c2a
  Preparing metadata (setup.py) ... done


In [49]:
from re import T
is_train_autoencoder = False
is_train_autoencoder_disable_iteration = False
is_train_mesh_transformer = True
is_clear_dataset_npz = False


In [50]:
import torch
import trimesh
import numpy as np
import os
import csv
from collections import OrderedDict

from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer
)
from meshgpt_pytorch.data import (
    derive_face_edges_from_faces
)

In [51]:
import trimesh
import torch
import numpy as np
import os
from collections import OrderedDict

max_faces = 4096

def get_mesh(file_path):
    mesh = trimesh.load(file_path, force='mesh')

    # Center and scale vertices
    center = np.mean(mesh.vertices, axis=0)
    vertices = mesh.vertices - center
    max_abs = np.max(np.abs(vertices))
    scale_factor = (1 / 128) / max_abs
    vertices *= scale_factor

    # Quantize vertices
    vertices = np.around(vertices).astype(np.float32)

    # Sort vertices by Z, Y, X
    sorted_indices = np.lexsort(vertices.T[::-1])
    vertices = vertices[sorted_indices]

    # Map old indices to new, sorted indices
    vertex_map = np.empty(len(sorted_indices), dtype=int)
    vertex_map[sorted_indices] = np.arange(len(sorted_indices))

    # Reindex faces
    reindexed_faces = vertex_map[mesh.faces]
    sorted_faces = np.sort(reindexed_faces, axis=1)

    return vertices, sorted_faces

def augment_mesh(vertices, jitter_strength=0.01):
    jitter_amount = np.random.uniform(-jitter_strength, jitter_strength, size=vertices.shape)
    vertices += jitter_amount
    return vertices

def snake_to_sentence_case(snake_str):
    components = snake_str.split("_")
    return " ".join(word.capitalize() for word in components)

def load_filename(directory, variations):
    obj_datas = []

    # Get random scale factors within a range
    scale_factors = np.random.uniform(0.75, 1.0, size=variations)

    for filename in os.listdir(directory):
        if filename.endswith(".glb"):
            file_path = os.path.join(directory, filename)
            vertices, faces = get_mesh(file_path)

            if len(faces) > max_faces:
                print(f"Mesh {filename} has {len(faces)} faces which is more than the allowed {max_faces} faces. Rejecting.")
                continue

            faces_tensor = torch.tensor(faces, dtype=torch.long).to("cuda")
            face_edges = derive_face_edges_from_faces(faces_tensor)

            text, _ = os.path.splitext(filename)
            text = snake_to_sentence_case(text)
            # Run video llava on the image. "Describe the focus of the photo as a json dictionary."
            for scale_factor in scale_factors:
                aug_vertices = augment_mesh(vertices.copy()) * scale_factor
                aug_vertices_tensor = torch.tensor(aug_vertices, dtype=torch.float)

            obj_data = {
                "vertices": aug_vertices_tensor.to("cuda"),
                "faces": faces_tensor,
                "face_edges": face_edges,
                "texts": text
            }
            obj_datas.append(obj_data)

    print(f"[create_mesh_dataset] Returning {len(obj_datas)} meshes")
    return obj_datas


In [71]:
from pathlib import Path
import gc
import torch
import os
from meshgpt_pytorch import MeshDataset

torch.cuda.empty_cache()
gc.collect()

project_name = "meshgpt-pytorch"

working_dir = f'/workspace/{project_name}/dataset/blockmesh_test'

working_dir = Path(working_dir)
working_dir.mkdir(exist_ok = True, parents = True)
dataset_path = working_dir / (project_name + ".npz")


if is_clear_dataset_npz:
    data = load_filename(working_dir, 1)
    dataset = MeshDataset(data)
    dataset.generate_face_edges()
    dataset.save(dataset_path)
else:
    dataset = MeshDataset.load(dataset_path)

[MeshDataset] Loaded 44 entrys
[MeshDataset] Created from 44 entrys


### Inspect

In [72]:
seen_texts = []
mesh_list = []  # List to store individual meshes
translation_distance = 1.0  # Distance to translate vertices, set to 1 unit as required

# Iterate over each item in the dataset
for r, item in enumerate(dataset.data):
    # Get vertices and faces
    vertices = np.array(item['vertices'].cpu())
    faces = np.array(item['faces'].cpu())
    texts = item['texts']

    if texts in seen_texts:
      continue
    seen_texts.append(texts)

    # Translate the vertices copy
    translation_vector = np.array([len(seen_texts) * translation_distance, 0, 0])  # Translation along the x-axis
    vertices[:, :3] += translation_vector  # Apply translation only to x, y, z

    # Create a new mesh object with the translated vertices and original faces
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)

    # Append the new mesh to our list of meshes
    mesh_list.append(mesh)

    print(f"Iteration {r} complete. Processed texts {texts} with {len(vertices)} vertices and {len(faces)} faces.")

# After iterating over all items, print the number of processed meshes
print(f"Total number of processed meshes: {len(mesh_list)}")


Iteration 0 complete. Processed texts Mire Clothing with 2179 vertices and 3884 faces.
Iteration 1 complete. Processed texts S Bed Full with 62 vertices and 48 faces.
Iteration 2 complete. Processed texts S Bed King with 403 vertices and 480 faces.
Iteration 3 complete. Processed texts S Bed Twin with 403 vertices and 480 faces.
Iteration 4 complete. Processed texts S Bone with 56 vertices and 24 faces.
Iteration 5 complete. Processed texts S Box with 24 vertices and 12 faces.
Iteration 6 complete. Processed texts S Cabinet Bookshelf with 690 vertices and 512 faces.
Iteration 7 complete. Processed texts S Cabinet Dresser 03 with 552 vertices and 256 faces.
Iteration 8 complete. Processed texts S Cabinet Dresser 05 with 744 vertices and 344 faces.
Iteration 9 complete. Processed texts S Chair Bar with 385 vertices and 440 faces.
Iteration 10 complete. Processed texts S Chair Box with 1848 vertices and 912 faces.
Iteration 11 complete. Processed texts S Chair Modern with 904 vertices and

### Train!

In [54]:
autoencoder = MeshAutoencoder().to("cuda")

**Have at least 400-2000 items in the dataset, use this to multiply the dataset**  

In [55]:
import random


random.seed(42)
random.shuffle(dataset.data)

initial_length = len(dataset.data)
target_length = 400
print(initial_length)
if initial_length > 0:
    replication_factor = max(1, (target_length - 1) // initial_length + 1)
    dataset.data *= replication_factor
    dataset.data = dataset.data[:target_length]

print(len(dataset.data))

44
2000


**Train to about 0.3 loss if you are using a small dataset**

In [56]:
autoencoder_trainer = MeshAutoencoderTrainer(model = autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                            batch_size=8,
                                            grad_accum_every=2,
                                            learning_rate = 4e-3)
if is_train_autoencoder:
  if not is_train_autoencoder_disable_iteration:
    autoencoder_trainer.load(f'{working_dir}/mesh-encoder_{project_name}.pt')
  loss = autoencoder_trainer.train(380,stop_at_loss = 0.28, diplay_graph= True)
  autoencoder_trainer.save(f'{working_dir}/mesh-encoder_{project_name}.pt')
else:
  autoencoder_trainer.load(f'{working_dir}/mesh-encoder_{project_name}.pt')
  autencoder = autoencoder_trainer.model
  for param in autoencoder.parameters():
      param.requires_grad = True
  import gc

In [57]:
import gc
torch.cuda.empty_cache()
gc.collect()

max_length =  max(len(d["faces"]) for d in dataset if "faces" in d)
max_seq = max_length * 6
print("Highest face count:" , max_length)
print("Max token sequence:" , max_seq)

transformer = MeshTransformer(
    autoencoder,
    dim = 512,
    coarse_pre_gateloop_depth = 6, # Better performance using more gateloop layers
    fine_pre_gateloop_depth= 4,
    #attn_depth = 24, # GPT-2 medium have 24 layer depth, change if needed
    max_seq_len = max_seq,
    condition_on_text = True,
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", ## Change or remove this line if you are using:  https://github.com/MarcusLoppe/classifier-free-guidance-pytorch
    text_condition_cond_drop_prob = 0.0
)

total_params = sum(p.numel() for p in transformer.decoder.parameters())
total_params = f"{total_params / 1000000:.1f}M"
print(f"Decoder total parameters: {total_params}")
total_params = sum(p.numel() for p in transformer.parameters())
total_params = f"{total_params / 1000000:.1f}M"
print(f"Total parameters: {total_params}")

Highest face count: 3884
Max token sequence: 23304


/root/micromamba/envs/py311_meshgpt/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Decoder total parameters: 94.4M
Total parameters: 152.6M


## **Required!**, embed the text and run generate_codes to save 4-96 GB VRAM (dependant on dataset) ##

**If you don't;** <br>
During each during each training step the autoencoder will generate the codes and the text encoder will embed the text.
<br>
After these fields are generate: **they will be deleted and next time it generates the code again:**<br>

This is due to the dataloaders nature, it writes this information to a temporary COPY of the dataset


In [58]:
labels = set(item["texts"] for item in dataset.data)
print(labels)
dataset.embed_texts(transformer)
dataset.generate_codes(autoencoder)
print(dataset.data[0].keys())

{'S Chair Sofa', 'S Chair Bar', 'S Hmd', 'S Table Sit Square', 'S Chair Stool Mini', 'Mire Clothing', 'S Primitive Pyramid', 'S Chair Sofa Wide', 'S Bed Twin', 'S Door Single Frame', 'S Door Double Frame', 'S Cabinet Bookshelf', 'S Primitive Wedge', 'S Table Bar Rectangle', 'S Chair Stool', 'S Box', 'S Cabinet Dresser 05', 'S Primitive Cylinder', 'S Table Bar Circle', 'S Gui', 'S Bed King', 'S Table Nightstand', 'S Table Bedside', 'S Stairs Single-6', 'S Tree No Leaves', 'S Phone', 'S Bone', 'S Chair Modern', 'S Table Office', 'S Door Single', 'S Cabinet Dresser 03', 'S Bed Full', 'S Primitive Cylinder Hollow', 'S Primitive Sphere', 'Sk Snake 01', 'S Chair Box', 'S Table Counter', 'Sk Cat 01', 'S Table Bar', 'S Table Sit Circle', 'S Tree Bushy', 'S Mask', 'S Table Sit Rectangle', 'S Ziggurat'}
[MeshDataset] Generated 44 text_embeddings
[MeshDataset] Generated codes for 2000 entrys
dict_keys(['vertices', 'faces', 'face_edges', 'text_embeds', 'codes'])


*Load previous saved model if you had to restart session*

**Train to about 0.0001 loss (or less) if you are using a small dataset**

TODO: Needs a fail early system.

In [59]:
steps = 100
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=4,num_train_steps=steps, dataset = dataset,
                                 learning_rate = 5e-4, batch_size=1)
if is_train_mesh_transformer:
  loss = trainer.train(steps, stop_at_loss = 0.02)
else:
  trainer.load(f'{working_dir}/mesh-transformer_{project_name}.pt')
transformer = trainer.model

Epoch 1/7: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [07:08<00:00,  4.66it/s, loss=7.4]


Epoch 1 average loss: 7.416166525118053


Epoch 2/7: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [07:09<00:00,  4.65it/s, loss=5.65]


Epoch 2 average loss: 4.328081157442182


Epoch 3/7: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [07:07<00:00,  4.68it/s, loss=0.0952]


...

Epoch 5 average loss: 0.26045236000115984          avg loss speed: 2.2376173476779364


Epoch 6/7: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [07:09<00:00,  4.65it/s, loss=0.0254]


Epoch 6 average loss: 0.054236595172202216          avg loss speed: 1.0879568466932203


Epoch 7/7: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [06:59<00:00,  4.76it/s, loss=0.00704]

Epoch 7 average loss: 0.012523111718823202          avg loss speed: 0.3954176315490234
Stopping training at epoch 6 with average loss 0.012523111718823202
Training complete


In [61]:
trainer.save(f'{working_dir}\mesh-transformer_{project_name}.pt')   

In [ ]:
trainer.save(f'{working_dir}\mesh-transformer_{project_name}.pt')   

**Load the latest model**

In [64]:
autoencoder_trainer = MeshAutoencoderTrainer(model = autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100, batch_size=1,  grad_accum_every=1, learning_rate = 1e-4)
autoencoder_trainer.load(f'{working_dir}/mesh-encoder_{project_name}.pt')
autencoder = autoencoder_trainer.model
for param in autoencoder.parameters():
    param.requires_grad = True
import gc
torch.cuda.empty_cache()
gc.collect()

max_length =  max(len(d["faces"]) for d in dataset if "faces" in d)
max_seq = max_length * 6
print("Highest face count:" , max_length)
print("Max token sequence:" , max_seq)

transformer = MeshTransformer(
    autoencoder,
    dim = 512,
    coarse_pre_gateloop_depth = 6, # Better performance using more gateloop layers
    fine_pre_gateloop_depth= 4,
    # attn_depth = 24, # GPT-2 medium have 24 layer depth, change if needed
    max_seq_len = max_seq,
    condition_on_text = True,
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", ## Change or remove this line if you are using:  https://github.com/MarcusLoppe/classifier-free-guidance-pytorch
    text_condition_cond_drop_prob = 0.0
).to("cuda")

trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=1,num_train_steps=100, dataset = dataset, learning_rate = 1e-1, batch_size=2)
trainer.load(f'{working_dir}\mesh-transformer_{project_name}.pt')
transformer = trainer.model


Highest face count: 3884
Max token sequence: 23304


## Generate and view mesh

In [66]:
def combind_mesh(path, mesh):
    all_vertices = []
    all_faces = []
    vertex_offset = 0
    translation_distance = 0.5

    for r, faces_coordinates in enumerate(mesh):
        numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)

        for vertex in numpy_data:
            all_vertices.append(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

        for i in range(1, len(numpy_data), 3):
            all_faces.append(f"f {i + vertex_offset} {i + 1 + vertex_offset} {i + 2 + vertex_offset}\n")

        vertex_offset += len(numpy_data)

    obj_file_content = "".join(all_vertices) + "".join(all_faces)

    with open(path , "w") as file:
        file.write(obj_file_content)

def combind_mesh_with_rows(path, meshes):
    all_vertices = []
    all_faces = []
    vertex_offset = 0
    translation_distance = 0.5

    for row, mesh in enumerate(meshes):
        for r, faces_coordinates in enumerate(mesh):
            numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)
            numpy_data[:, 0] += translation_distance * (r / 0.2 - 1)
            numpy_data[:, 2] += translation_distance * (row / 0.2 - 1)

            for vertex in numpy_data:
                all_vertices.append(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

            for i in range(1, len(numpy_data), 3):
                all_faces.append(f"f {i + vertex_offset} {i + 1 + vertex_offset} {i + 2 + vertex_offset}\n")

            vertex_offset += len(numpy_data)

        obj_file_content = "".join(all_vertices) + "".join(all_faces)

    with open(path , "w") as file:
        file.write(obj_file_content)


def write_mesh_output(path, coords):
    numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)
    obj_file_content = ""

    for vertex in numpy_data:
        obj_file_content += f"v {vertex[0]} {vertex[1]} {vertex[2]}\n"

    for i in range(1, len(numpy_data), 3):
        obj_file_content += f"f {i} {i + 1} {i + 2}\n"

    with open(path, "w") as file:
        file.write(obj_file_content)


**Using only text**

In [76]:

from pathlib import Path

folder = working_dir / 'renders'
obj_file_path = Path(folder)
obj_file_path.mkdir(exist_ok = True, parents = True)

text_coords = []
for text in labels:
    print(f"Generating {text}")
    faces_coordinates = transformer.generate(texts = [text],  temperature = 0.0)
    text_coords.append(faces_coordinates)

    write_mesh_output(f'{folder}/3d_output_{text}.obj', faces_coordinates)


combind_mesh(f'{folder}/3d_models_all.obj', text_coords)

Generating S Chair Sofa


 43%|██████████████████████████████████████████████████████████████████████████▋                                                                                                  | 10056/23304 [01:08<01:30, 146.50it/s]


Generating S Chair Bar


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23304/23304 [02:39<00:00, 146.25it/s]


Generating S Hmd


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23304/23304 [02:38<00:00, 147.03it/s]


Generating S Table Sit Square


 12%|████████████████████▏                                                                                                                                                    | 2784/23304 [00:18<02:17, 149.66it/s]

# **Preview 3d Models**

In [ ]:
!pip install ipyvolume trimesh pythreejs
import trimesh

folder = working_dir / 'renders'

mesh = trimesh.load(f'{folder}/3d_models_all.obj')
import ipyvolume as ipv

vertices = mesh.vertices
faces = mesh.faces

fig = ipv.figure()
ipv.plot_trisurf(vertices[:,0], vertices[:,1], vertices[:,2], triangles=faces)
ipv.show()

**Text + prompt of tokens**

Grab fresh copy of dataset

In [ ]:
dataset = MeshDataset.load(dataset_path)
dataset.generate_codes(autoencoder)

**Prompt with 10% of codes/tokens BROKEN**

In [68]:
from pathlib import Path
token_length_procent = 0.10
codes = []
texts = []
for label in labels:
    for item in dataset.data:
        if item['texts'] == label:
            num_tokens = int(item["codes"].shape[0] * token_length_procent)

            texts.append(item['texts'])
            codes.append(item["codes"].flatten()[:num_tokens].unsqueeze(0))
            break

folder = working_dir / f'renders/text+codes'
obj_file_path = Path(folder)
obj_file_path.mkdir(exist_ok = True, parents = True)

coords = []



for text, prompt in zip(texts, codes):
    print(f"Generating {text} with {prompt.shape[1]} tokens")
    faces_coordinates = transformer.generate(texts = [text],  prompt = prompt, temperature = 0)
    coords.append(faces_coordinates)

    obj_file_path = f'{folder}/{text}_{prompt.shape[1]}_tokens.obj'
    write_mesh_output(obj_file_path, faces_coordinates)

    print(obj_file_path)


combind_mesh(f'{folder}/text+prompt_all.obj', coords)

if text_coords is not None:
    combind_mesh_with_rows(f'{folder}/both_verisons.obj', [text_coords , coords])

KeyError: 'texts'

**Prompt with 0% to 80% of tokens BROKEN**

In [74]:
import json
import numpy as np
import trimesh
from pathlib import Path

def convert_to_obj(vertices, faces, output_file_path):
    scene = trimesh.Scene()
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    scene.add_geometry(mesh)

    with open(output_file_path, "w") as f:
        f.write(scene.export(file_type="obj"))

def encode_to_pua(codes):
    flat_codes = [item for sublist in codes for subsublist in sublist for item in subsublist]
    return "".join(chr(code + 0xF0000) for code in flat_codes)

jsonl_lines = []

for token_length_procent in np.arange(0, 0.8, 0.1):
    codes = []
    texts = []
    for label in labels:
        for item in dataset.data:
            if item['texts'] == label:
                num_tokens = int(item["codes"].shape[0] * token_length_procent)

                texts.append(item['texts'])
                codes.append(item["codes"].flatten()[:num_tokens].unsqueeze(0))
                break

    coords = []
    for text, code in zip(texts, codes):
        print(f"Generating {text} with {code.shape[1]} tokens")
        faces_coordinates = transformer.generate(texts=[text], prompt=code, temperature=0)
        coords.append(faces_coordinates)

        # Process mesh data inlined here
        continuous_coors_list = [np_array.tolist() for np_array in faces_coordinates]
        flat_list = [item for sublist in continuous_coors_list for item in sublist]
        vertices = [vertex for sublist in flat_list for vertex in sublist]
        faces = [[i, i + 1, i + 2] for i in range(0, len(vertices), 3)]

        obj_filename = f'{text}_{code.shape[1]}_tokens.obj'
        obj_file_path = folder / obj_filename
        convert_to_obj(vertices, faces, obj_file_path)

        encoded_codes = encode_to_pua(code.cpu().tolist())

        with open(obj_file_path, "r") as file:
            obj_contents = file.read()

        # Append line to JSONL structure
        jsonl_line = [
            {"role": "system", "content": "This assistant can understand 3D models using the meshgpt-pytorch Unicode plane 15 16384 item codebook for triangles and the .obj 3d format."},
            {"role": "user", "content": encoded_codes},
            {"role": "assistant", "content": obj_contents}
        ]
        jsonl_lines.append(jsonl_line)

        print(obj_file_path)

    mesh_rows.append(coords)
    combind_mesh(f'{folder}/text+prompt_all_{token_length_procent}.obj', coords)

combind_mesh_with_rows(f'{folder}/all.obj', mesh_rows)

with open("chatml.jsonl", "w", encoding="utf-8") as f:
    for item in jsonl_lines:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

KeyError: 'codes'